<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Turkish/base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turkish Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Turkish


In [3]:
ls

cc.tr.300.vec     offenseval-annotation.txt      readme-trainingset-tr.txt
cc.tr.300.vec.gz  offenseval-tr-training-v1.tsv


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [5]:
train.head()

,id,tweet,subtask_a
19063,23945,Araplarda hattâ Afrikada bile böyle sıkıntı yo...,OFF
22700,32971,@USER Çay ocağından yemek menüsü öğrenmek 🤔 ga...,NOT
15065,44980,oraya buraya fotonuzu atmayın işte benim kanki...,OFF
19493,46065,Herşeyin farkında olup birşey yapamayan insan ...,NOT
14575,21239,@USER Sayın müdürüm 2017 kpss sizden adil çözü...,NOT


In [6]:
train.tail()

,id,tweet,subtask_a
29243,46053,@USER @USER @USER İş olsa da uyumuyoruz ki.. 😂,NOT
29762,24356,@USER Benim de öyle ve film efsane gitmediysen...,NOT
17982,22061,Kadınlar maçı tekrar olsa keşke be,NOT
12052,42625,@USER @USER @USER @USER @USER @USER @USER @USE...,NOT
15649,10397,"Flaka at, Zombi ol Gel kollarıma. Miras...",NOT


In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [8]:
import collections
collections.Counter(labels)

Counter({0: 6046, 1: 25231})

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train,labels,test_size=0.2)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test  = vectorizer.transform(X_test)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report,f1_score

classifier = LogisticRegression(solver='lbfgs',max_iter=500)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(f1_score(y_test, y_pred,average='macro'))


0.6880333471113433


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report,f1_score

classifier = SVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(f1_score(y_test, y_pred,average='macro'))


0.5635079691292593


# fine tuning

In [0]:
vectorizer = CountVectorizer()
vectorizer.fit(train)
features = vectorizer.transform(train)

In [0]:
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y, nfolds):
    Cs = [ 0.1, 1, 10,15,20,25,30,40,50,60,70,100,120]
    gammas = [0.0001,0.0005,0.001,0.005,0.008, 0.01, 0.1,0.3,0.5, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds,scoring='f1')
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    return grid_search.best_score_


svc_param_selection(features,labels,10)

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
def logistic_param_selection(X, y, nfolds):
    C= [0.1, 1,3,5,8, 10,12,15]
    param_grid = {'C': C}
    grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=nfolds,scoring='f1_macro')
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    return grid_search.best_score_

logistic_param_selection(features,labels,10)

{'C': 12}


0.7085723696945098